In [24]:
client = MongoClient()
db = client.openfoodfacts
collection = db.products

In [74]:
c_f = collection.find().limit(2)
data = pd.DataFrame(list(c_f))

,_id,popularity_key,nutrition_score_warning_fruits_vegetables_nuts_estimate_from_ingredients,unknown_ingredients_n,ingredients_analysis_tags,serving_size,checkers_tags,countries_hierarchy,_keywords,ingredients_n_tags,...,categories_properties,ingredients_text_with_allergens_en,ingredients_text_is,generic_name_is,ingredients_text_with_allergens_is,product_name_is,generic_name_en,nutrition_score_debug,product_name_en,ingredients_text_en
0,3261830123377,100000000008,1.0,3.0,"[en:palm-oil-content-unknown, en:non-vegan, en...",40g,[],[en:france],"[colorant, san, diverse, viande, qualite, de, ...","[12, 11-20]",...,{'ciqual_food_code:en': '8332'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5690522007516,-199999999992,NaN,NaN,NaN,NaN,[],[en:iceland],[],NaN,...,NaN,,,,,,,no score when the product does not have a cate...,,


In [749]:
proj = { 
    'product_name_fr' : 1, 
    'ingredients' : 1, 
    'salt_content' : 1, 
    'oil_content' : 1, 
    'product_name' : 1,
    'nutriments' : 1,
    'compared_to_category' : 1,
    'interface_version_modified' : 1
}

addFields = { 
    
    'ingredients' : "$ingredients.text",
    
    'salt_content' : {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ 
                   { '$eq' : [ "$$item.text", 'Sel' ] }, 
                   { '$eq' : [ "$$item.text", 'sel' ] },
                   { '$eq' : [ "$$item.text", 'SEL' ] },
                   { '$eq' : [ "$$item.text", 'VINAGE SEL' ] },
                   { '$eq' : [ "$$item.text", 'sel marin' ] },
                   { '$eq' : [ "$$item.text", 'Sel marin' ] },
                   { '$eq' : [ "$$item.text", 'salt' ] },
                   { '$eq' : [ "$$item.text", 'Salt' ] }] }
            }
    },
    
    
    'oil_content': {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile extra vierge d\'olive' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra-vierge' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile dolive* vierge exta' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'huile de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extra extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'huile de tournesol désodorisée' ] }, {
                   '$eq' : [ "$$item.text", 'Extra Virgin Olive Oil' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile de palme' ] }, {
                   '$eq' : [ "$$item.text", 'natives Olivenöl extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Sonnenblumenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Olivenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile Végétale' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de colza' ] }, {
                   '$eq' : [ "$$item.text", 'huile végétale de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile végétale de colza' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] } ] 
                    }
            }
    },
}




match = {
    
    'ingredients' : { 
        "$elemMatch" : { 
            "text" : { '$regex' : "tomate*", '$options' : "i" },
        } 
    },
    
    "product_name_fr" : { '$regex' : "sauce*", '$options' : "i" }
    
}



c_f = collection.aggregate([
    { '$match' : match } ,
    { '$project' : proj },
    { '$addFields' : addFields }
])

data = pd.DataFrame(list(c_f))

In [695]:
data.head(10)

,_id,ingredients,product_name_fr,salt_content,oil_content
0,0000069162176,"[INGREDIENTES, agua, harina de tngo tierno car...",Quenelles de brochet sauce Nantua,[],[]
1,00005010,"[Tomato puree, beef mince meat, carrots, extra...",Organic tomato sauce bolognese,[],[]
2,00021036,"[sauce à la tomate, à Itoignon kJ et à la caro...",Tomato & Gorgonzola pasta sauce,[],[]
3,00089227,"[Pulpe de tomate, Saucisse de porc, Concentré ...",Tomato & Sausage Pasta Sauce,"[{'id': 'en:salt', 'vegan': 'yes', 'text': 'Se...","[{'from_palm_oil': 'no', 'id': 'en:extra-virgi..."
4,0011110791474,"[Water, red bell peppers, distilled vinegar, t...",Peri peri chili culinary hot sauce,[],[]
5,0013000009638,"[concentré de tormate, isoglucose, vinaigre bl...",Sauce barbecue,"[{'rank': 6, 'text': 'sel', 'percent_min': 0, ...",[]
6,0013000526807,"[tomato puree, high fructose corn syrup, disti...",Heinz 57 Sauce,[],[]
7,0013409515518,"[High fructose corn syrup, distilled vinegar, ...",Sweet 'n Spicy Barbecue Sauce,[],[]
8,0016229906948,"[EAU, ANANAS, SUCRE, PUREEDE TOMATE, AMIDON DE...",Sauce Aigre Douce 720 ML,[],[]
9,0016229910051,"[EAU, ANANAS, SUCRE, PUREE DE TOMATE, AMIDON D...",Sauce Aigre Douce 280ML,[],[]


In [710]:
data.iloc[428]['ingredients']

['Lentils',
 'Toulouse sausages',
 'stock',
 'water',
 'carrots',
 'onions duck fat',
 'sait',
 't',
 'pork meat',
 'salt',
 'pepper',
 'natural pork casing',
 'water',
 'pork bones and pork rind',
 'tomato concentrate',
 'salt']

In [750]:
max_salt_content = []
for l in data['salt_content'] :
    if len(l) > 0 and 'percent_max' in l[0]  :
        max_salt_content.append(l[0]['percent_max'])
    else :
        max_salt_content.append(0)
        
data['max_salt_content'] = max_salt_content

min_salt_content = []
for l in data['salt_content'] :
    if len(l) > 0 and 'percent_min' in l[0]  :
        min_salt_content.append(l[0]['percent_min'])
    else :
        min_salt_content.append(0)
        
data['min_salt_content'] = min_salt_content

max_oil_content = []
for l in data['oil_content'] :
    if len(l) > 0 and 'percent_max' in l[0]  :
        max_oil_content.append(l[0]['percent_max'])
    else :
        max_oil_content.append(0)
        
data['max_oil_content'] = max_oil_content

min_oil_content = []
for l in data['oil_content'] :
    if len(l) > 0 and 'percent_min' in l[0]  :
        min_oil_content.append(l[0]['percent_min'])
    else :
        min_oil_content.append(0)
        
data['min_oil_content'] = min_oil_content

oil_type = []
for l in data['oil_content'] :
    if len(l) > 0 and 'percent_min' in l[0]  :
        oil_type.append(l[0]['text'])
    else :
        oil_type.append("")
        
data['oil_type'] = oil_type

In [751]:
data

,_id,nutriments,product_name,interface_version_modified,compared_to_category,ingredients,product_name_fr,salt_content,oil_content,max_salt_content,min_salt_content,max_oil_content,min_oil_content,oil_type
0,0000069162176,"{'salt_unit': '', 'energy': 586, 'proteins': 4...",Quenelles de brochet sauce Nantua,20150316.jqm2,en:pike-quenelles,"[INGREDIENTES, agua, harina de tngo tierno car...",Quenelles de brochet sauce Nantua,[],[],0,0,0,0,
1,00005010,"{'energy-kcal_value': 106, 'sodium_100g': 0.16...",Organic tomato sauce bolognese,20150316.jqm2,en:bolognese-sauces,"[Tomato puree, beef mince meat, carrots, extra...",Organic tomato sauce bolognese,"[{'id': 'en:salt', 'text': 'salt', 'vegetarian...",[],1.02307,0.0826429,0,0,
2,00021036,"{'salt': 0.6, 'fiber_100g': 3.1, 'carbohydrate...",Tomato & Gorgonzola pasta sauce,20150316.jqm2,NaN,"[sauce à la tomate, à Itoignon kJ et à la caro...",Tomato & Gorgonzola pasta sauce,[],[],0,0,0,0,
3,00089227,"{'fiber': 1.7, 'fruits-vegetables-nuts-estimat...",Tomato & Sausage Pasta Sauce,20120622,en:tomato-sauces,"[Pulpe de tomate, Saucisse de porc, Concentré ...",Tomato & Sausage Pasta Sauce,"[{'id': 'en:salt', 'vegan': 'yes', 'text': 'Se...","[{'from_palm_oil': 'no', 'id': 'en:extra-virgi...",0,0,0,0,
4,0011110791474,"{'sodium_100g': 2, 'energy-kcal_value': 100, '...",Peri peri chili culinary hot sauce,20150316.jqm2,en:sauces,"[Water, red bell peppers, distilled vinegar, t...",Peri peri chili culinary hot sauce,"[{'percent_min': 0, 'rank': 6, 'percent_max': ...",[],16.6667,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2702,93201834,"{'energy-kcal_unit': 'kcal', 'sugars': 46, 'pr...",BBQ Sauce,20150316.jqm2,en:barbecue-sauces,"[sugar, water, white vinegar, tomato paste mal...",BBQ Sauce,"[{'vegetarian': 'yes', 'percent_max': 16.66666...",[],16.6667,0,0,0,
2703,9300681008403,"{'fat_unit': 'g', 'nova-group_serving': 4, 'sa...",Sweet chili sauce,20150316.jqm2,fr:sauces-tomates-pimentees,"[Sucre, eau, concentré de tomates, vinaigre, l...",Sweet chili sauce,"[{'text': 'sel', 'id': 'en:salt', 'percent_min...",[],2.5,0,0,0,
2704,9556041601214,"{'carbohydrates_unit': '', 'saturated-fat_100g...",Ayam Sardines In Tomato Sauce,20150316.jqm2,en:sardines-in-tomato-sauce,"[POISSON DE TYPE SARDINES, SAUCE TOMATE, PILCH...",Ayam Sardines In Tomato Sauce,"[{'vegan': 'yes', 'vegetarian': 'yes', 'text':...",[],0,0,0,0,
2705,9556041611121,"{'sodium': 1.08, 'proteins_serving': 0, 'salt'...",Sauce aigre-douce Ayam™,20190830,en:sweet-and-sour-sauces,"[EAU, SUCRE, ÉPAISSISSANT, CONCENTRÉ DE TOMATE...",Sauce aigre-douce Ayam™,"[{'percent_min': 0, 'vegetarian': 'yes', 'vega...",[],16.6667,0,0,0,


In [ ]:
oil_type = []
for l in data['oil_content'] :
    if len(l) > 0 and 'percent_min' in l[0]  :
        oil_type.append(l[0]['text'])
    else :
        oil_type.append("")
        
data['oil_type'] = oil_type

In [753]:
data['nutriments'][0]

{'salt_unit': '',
 'energy': 586,
 'proteins': 4.6,
 'energy_100g': 586,
 'proteins_value': 4.6,
 'proteins_unit': '',
 'fiber': 0.5,
 'nutrition-score-fr': 16,
 'sodium_unit': 'g',
 'salt_value': 4.9,
 'fiber_100g': 0.5,
 'energy-kcal_value': 140,
 'carbohydrates_100g': 5.1,
 'salt': 4.9,
 'nutrition-score-fr_serving': 16,
 'nova-group': 4,
 'fiber_value': 0.5,
 'fat': 11,
 'carbohydrates': 5.1,
 'energy_value': 140,
 'sodium_value': 1.9600000000000002,
 'fruits-vegetables-nuts-estimate-from-ingredients_100g': 0,
 'sugars_100g': 0.6,
 'saturated-fat_value': 5.5,
 'sugars': 0.6,
 'fat_unit': '',
 'saturated-fat_unit': '',
 'saturated-fat_100g': 5.5,
 'sugars_unit': '',
 'carbohydrates_value': 5.1,
 'nova-group_serving': 4,
 'salt_100g': 4.9,
 'carbohydrates_unit': '',
 'energy_unit': 'kcal',
 'sodium_100g': 1.9600000000000002,
 'fiber_unit': '',
 'sugars_value': 0.6,
 'proteins_100g': 4.6,
 'fat_value': 11,
 'nova-group_100g': 4,
 'nutrition-score-fr_100g': 16,
 'energy-kcal': 140,
 'f